In [1]:
from ultralytics import YOLO
import os
import cv2
import shutil
from sklearn.model_selection import train_test_split
import numpy as np

In [59]:
IMAGES_DIR = '/home/jovyan/nazar/123/123/misis_chill/train_dataset/cv_open_dataset/open_img'  # Путь к вашему датасету с изображениями
MASKS_DIR = '/home/jovyan/nazar/123/123/misis_chill/train_dataset/cv_open_dataset/open_msk'  # Путь к вашему датасету с масками
OUTPUT_DIR = './datasets/train_data'  # Путь к выходной директории
TRAIN_SIZE = 0.8  # Процент обучающей выборки

In [60]:
os.makedirs(os.path.join(OUTPUT_DIR, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/val'), exist_ok=True)

In [61]:
image_files = [f for f in os.listdir(IMAGES_DIR) if f.endswith(('.jpg', '.png'))]
mask_files = [f for f in os.listdir(MASKS_DIR) if f.endswith('.png')]
if len(image_files) != len(mask_files):
    print(len(image_files))
    print(len(mask_files))
    print("Количество изображений и масок не совпадает.")
    for mask in mask_files:
        if mask.replace(".png", ".jpg") not in image_files:
            print(mask)

In [62]:
IMAGES_DIR2 = '/home/jovyan/nazar/123/123/misis_chill/train_dataset/cv_synt_dataset/synt_img'  # Путь к вашему датасету с изображениями
MASKS_DIR2 = '/home/jovyan/nazar/123/123/misis_chill/train_dataset/cv_synt_dataset/synt_msk'

In [63]:
image_files += ["synt" + f for f in os.listdir(IMAGES_DIR2) if f.endswith(('.jpg', '.png'))]
mask_files += [f for f in os.listdir(MASKS_DIR2) if f.endswith('.png')]
if len(image_files) != len(mask_files):
    print(len(image_files))
    print(len(mask_files))
    print("Количество изображений и масок не совпадает.")
    for mask in mask_files:
        if mask.replace(".png", ".jpg") not in image_files:
            print(mask)

In [64]:
image_files

['1710275253_0.jpg',
 '1710277054_0.jpg',
 '1710322071_0.jpg',
 '1710370672_0.jpg',
 '1710381477_0.jpg',
 '1710390473_0.jpg',
 '1710399478_0.jpg',
 '1710430072_0.jpg',
 '1710458871_0.jpg',
 '1710584874_0.jpg',
 '1710604665_0.jpg',
 '1710809853_0.jpg',
 '1710811652_0.jpg',
 '1710813452_0.jpg',
 '1710815253_0.jpg',
 '1710835053_0.jpg',
 '1710847680_0.jpg',
 '1710917853_0.jpg',
 '1711009681_0.jpg',
 '1711090673_0.jpg',
 '1711119479_0.jpg',
 '1711177052_0.jpg',
 '1711178852_0.jpg',
 '1711178872_0.jpg',
 '1711180652_0.jpg',
 '1711182453_0.jpg',
 '1711184253_0.jpg',
 '1711186053_0.jpg',
 '1711187868_0.jpg',
 '1711256271_0.jpg',
 '1711268872_0.jpg',
 '1711268876_0.jpg',
 '1711330053_0.jpg',
 '1711393084_0.jpg',
 '1711412853_0.jpg',
 '1711421860_0.jpg',
 '1711430853_0.jpg',
 '1711448853_0.jpg',
 '1711450664_0.jpg',
 '1711456067_0.jpg',
 '1711459680_0.jpg',
 '1711510053_0.jpg',
 '1712555862_0.jpg',
 '1712564893_0.jpg',
 '1712577488_0.jpg',
 '1712741269_0.jpg',
 '1712788070_0.jpg',
 '1713302111_

In [77]:
mask_files

['1710275253_0.png',
 '1710277054_0.png',
 '1710322071_0.png',
 '1710370672_0.png',
 '1710381477_0.png',
 '1710390473_0.png',
 '1710399478_0.png',
 '1710430072_0.png',
 '1710458871_0.png',
 '1710584874_0.png',
 '1710604665_0.png',
 '1710809853_0.png',
 '1710811652_0.png',
 '1710813452_0.png',
 '1710815253_0.png',
 '1710835053_0.png',
 '1710847680_0.png',
 '1710917853_0.png',
 '1711009681_0.png',
 '1711090673_0.png',
 '1711119479_0.png',
 '1711177052_0.png',
 '1711178852_0.png',
 '1711178872_0.png',
 '1711180652_0.png',
 '1711182453_0.png',
 '1711184253_0.png',
 '1711186053_0.png',
 '1711187868_0.png',
 '1711256271_0.png',
 '1711268872_0.png',
 '1711268876_0.png',
 '1711330053_0.png',
 '1711393084_0.png',
 '1711412853_0.png',
 '1711421860_0.png',
 '1711430853_0.png',
 '1711448853_0.png',
 '1711450664_0.png',
 '1711456067_0.png',
 '1711459680_0.png',
 '1711510053_0.png',
 '1712555862_0.png',
 '1712564893_0.png',
 '1712577488_0.png',
 '1712741269_0.png',
 '1712788070_0.png',
 '1713302111_

In [76]:
len((mask_files))

317

In [66]:
train_images, val_images = train_test_split(image_files, train_size=TRAIN_SIZE, random_state=42)

In [67]:
def convert_mask_to_yolo(mask_path, out):
    # Открываем изображение
    image = cv2.imread(mask_path)

    if image is None:
        print(f"Не удалось открыть изображение: {mask_path}")
        return
    
    height, width = image.shape[:2]

    # Создаем маску для черного цвета
    black_mask = cv2.inRange(image, (0, 0, 0), (50, 50, 50))

    # Создаем новое изображение, где черный цвет остается, а остальные цвета становятся белыми
    new_image = np.ones_like(image) * 255  # Начинаем с белого изображения
    new_image[black_mask > 0] = [0, 0, 0]  # Заменяем черные пиксели

    # Преобразуем в градации серого для нахождения контуров
    gray_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    # Находим контуры
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Сохраняем контуры в текстовом формате
    output_file_name = os.path.splitext(os.path.basename(mask_path))[0] + '.txt'
    output_file_path = os.path.join(OUTPUT_DIR, out, output_file_name)

    with open(output_file_path, 'w') as f:
        for index, contour in enumerate(contours):
            # Получаем координаты всех точек контура
            contour_points = contour.reshape(-1, 2)
            # Нормализуем координаты
            normalized_points = [(x / width, y / height) for x, y in contour_points]
            points_str = ' '.join(f"{x:.3f} {y:.3f}" for x, y in normalized_points)
            f.write(f"0 {points_str}\n")

In [68]:
# Копирование изображений и масок в соответствующие папки
for img in train_images:
    if 'synt' not in img:
        shutil.copy(os.path.join(IMAGES_DIR, img), os.path.join(OUTPUT_DIR, 'images/train', img))
        mask_name = img.replace('.jpg', '.png')
        convert_mask_to_yolo(os.path.join(MASKS_DIR, mask_name), 'labels/train')

In [42]:
train_images

['F1_1_1_2.ts_f_500.jpg',
 'F4_1_1_1.ts_f_500.jpg',
 '1710430072_0.jpg',
 'F5_1_2_1.ts_f_1000.jpg',
 'syntF7_1_1_1.ts_f_500.jpg',
 'syntF4_2_3_2.ts_f_500.jpg',
 'syntF4_2_2_2.ts_f_500.jpg',
 'F1_1_5_2.ts_f_1000.jpg',
 'F5_2_1_1.ts_f_1000.jpg',
 'synt1711187868_0.jpg',
 'syntF1_2_3_1.ts_f_500.jpg',
 '1717063048_0.jpg',
 'F2_2_1_1.ts_f_1000.jpg',
 'syntF2_2_1_1.ts_f_500.jpg',
 'F1_2_5_2.ts_f_500.jpg',
 'synt1711256271_0.jpg',
 'F5_2_1_2.ts_f_1000.jpg',
 '1710917853_0.jpg',
 '1711180652_0.jpg',
 'syntF2_2_2_1.ts_f_1000.jpg',
 'F5_1_3_2.ts_f_1000.jpg',
 'synt1715112967_0.jpg',
 'syntF1_2_2_1.ts_f_500.jpg',
 'syntF2_1_1_2.ts_f_500.jpg',
 'synt1711510053_0.jpg',
 'syntF1_1_5_2.ts_f_1000.jpg',
 'F5_2_2_2.ts_f_500.jpg',
 'synt1711456067_0.jpg',
 '1711090673_0.jpg',
 'synt1711180652_0.jpg',
 'F1_2_2_1.ts_f_1000.jpg',
 'syntF5_2_2_2.ts_f_1000.jpg',
 '1711009681_0.jpg',
 'F5_1_2_2.ts_f_500.jpg',
 'syntF5_2_2_2.ts_f_500.jpg',
 '1711268876_0.jpg',
 'syntF1_1_3_1.ts_f_500.jpg',
 'syntF1_1_1_1.ts_f_5

In [69]:
for synt_img in train_images:
    img = synt_img[4:]
    if 'synt' in synt_img:
        shutil.copy(os.path.join(IMAGES_DIR2, img), os.path.join(OUTPUT_DIR, 'images/train', img))
        mask_name = img.replace('.jpg', '.png')
        convert_mask_to_yolo(os.path.join(MASKS_DIR2, mask_name), 'labels/train')

In [70]:
for img in val_images:
    if 'synt' not in img:
        shutil.copy(os.path.join(IMAGES_DIR, img), os.path.join(OUTPUT_DIR, 'images/val', img))
        mask_name = img.replace('.jpg', '.png')
        convert_mask_to_yolo(os.path.join(MASKS_DIR, mask_name), 'labels/val')


In [78]:
val_images

['F1_1_4_2.ts_f_1000.jpg',
 'synt4_1709186522_0.png',
 '1711182453_0.jpg',
 'synt415_1709265902_0.png',
 '1710584874_0.jpg',
 'F2_2_1_1.ts_f_1000.jpg',
 'synt10_1709255042_0.png',
 'synt20_1709344621_0.png',
 'F1_1_1_1.ts_f_1000.jpg',
 'F4_1_1_2.ts_f_500.jpg',
 'synt3_1709116441_0.png',
 'F1_1_5_1.ts_f_1000.jpg',
 'F7_1_2_2.ts_f_500.jpg',
 'synt105_1712788070_0.png',
 'synt274_1709339881_0.png',
 'F1_2_3_1.ts_f_500.jpg',
 'synt1_1709104681_0.png',
 'F4_2_3_2.ts_f_500.jpg',
 'synt96_1710262654_0.png',
 'synt406_1709191202_0.png',
 'synt267_1709260681_0.png',
 'synt11_1709260082_0.png',
 'synt262_1709232001_0.png',
 'F2_2_2_2.ts_f_500.jpg',
 '1712555862_0.jpg',
 'F4_1_2_1.ts_f_500.jpg',
 'synt79_1709260681_0.png',
 '1711393084_0.jpg',
 'F4_1_1_1.ts_f_500.jpg',
 'synt16_1709265962_0.png',
 'F4_2_2_1.ts_f_500.jpg',
 '1710390473_0.jpg',
 'F7_2_1_2.ts_f_1000.jpg',
 'synt40_1717060391_0.png',
 'synt81_1709262422_0.png',
 'F1_1_5_2.ts_f_1000.jpg',
 'synt107_1717060391_0.png',
 'synt25_17098882

In [79]:
for synt_img in val_images:
    img = synt_img[4:]
    if 'synt' in synt_img:
        shutil.copy(os.path.join(IMAGES_DIR2, img), os.path.join(OUTPUT_DIR, 'images/val', img))
        mask_name = img.replace('.jpg', '.png')
        convert_mask_to_yolo(os.path.join(MASKS_DIR2, mask_name), 'labels/val')


In [80]:
data_yaml_content = f"""
train: train_data/images/train
val: train_data/images/val

nc: 1  # Обновите количество классов (1 для загрязнения)
names: ['contaminated']  # Обновите названия классов
"""

with open('data.yaml', 'w') as f:
    f.write(data_yaml_content)

print("Датасет успешно разбит и сохранен в структуре проекта.")

Датасет успешно разбит и сохранен в структуре проекта.


In [81]:
model = YOLO("yolo11n-seg.pt")

In [82]:
train_results = model.train(
    data="./data.yaml",  # path to dataset YAML
    epochs=20,  # number of training epochs
    imgsz=640,  # training image size
    device="0",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
    
)

Ultralytics 8.3.44 🚀 Python-3.10.15 torch-2.5.1+cu124 CUDA:0 (A100 80GB PCIe, 81252MiB)
engine/trainer: task=segment, mode=train, model=yolo11n-seg.pt, data=./data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True,

train: Scanning /home/jovyan/nazar/123/123/misis_chill/train_dataset/baseline/datasets/train_data/labels/train... 288 images, 126 backgrounds, 0 corrupt: 100%|██████████| 288/288 [00:00<00:00, 512.34it/s]

train: New cache created: /home/jovyan/nazar/123/123/misis_chill/train_dataset/baseline/datasets/train_data/labels/train.cache



val: Scanning /home/jovyan/nazar/123/123/misis_chill/train_dataset/baseline/datasets/train_data/labels/val... 111 images, 64 backgrounds, 0 corrupt: 100%|██████████| 111/111 [00:00<00:00, 510.40it/s]

val: New cache created: /home/jovyan/nazar/123/123/misis_chill/train_dataset/baseline/datasets/train_data/labels/val.cache


...

      20/20      3.68G     0.8975      1.382      0.867     0.9709        165        640: 100%|██████████| 18/18 [00:01<00:00,  9.43it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  7.76it/s]

                   all        111        479      0.886      0.789      0.832      0.609      0.889      0.768      0.811      0.516



20 epochs completed in 0.020 hours.
Optimizer stripped from runs/segment/train5/weights/last.pt, 6.0MB
Optimizer stripped from runs/segment/train5/weights/best.pt, 6.0MB

Validating runs/segment/train5/weights/best.pt...
Ultralytics 8.3.44 🚀 Python-3.10.15 torch-2.5.1+cu124 CUDA:0 (A100 80GB PCIe, 81252MiB)
YOLO11n-seg summary (fused): 265 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.51it/s]


                   all        111        479      0.886      0.789      0.832       0.61      0.883      0.771      0.811      0.517
Speed: 0.2ms preprocess, 0.9ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/segment/train5


In [23]:
results = model.val(data="./data.yaml")
print(results)

Ultralytics 8.3.44 🚀 Python-3.10.15 torch-2.5.1+cu124 CUDA:0 (A100 80GB PCIe, 81252MiB)
YOLO11n-seg summary (fused): 265 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs


val: Scanning /home/jovyan/nazar/123/123/misis_chill/train_dataset/baseline/datasets/train_data/labels/val.cache... 62 images, 46 backgrounds, 0 corrupt: 100%|██████████| 62/62 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.67it/s]


                   all         62         33      0.829      0.545       0.63      0.489      0.829      0.545      0.602      0.436
Speed: 0.2ms preprocess, 5.3ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/segment/train42
ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f2a34133580>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,  

In [15]:
def infer_image(image_path):
    # Загрузка изображения
    image = cv2.imread(image_path)

    # Инференс
    return model(image)



In [16]:
# Функция для создания маски с черным фоном
def create_mask(image_path, results):
    # Загружаем изображение и переводим в градации серого
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    # Создаем пустую маску с черным фоном
    mask = np.zeros((height, width), dtype=np.uint8)

    # Проходим по результатам и создаем маску
    for result in results:
        masks = result.masks  # Получаем маски из результатов
        if masks is not None:
            for mask_array in masks.data:  # Получаем маски как массивы
                mask_i = mask_array.numpy()  # Преобразуем маску в numpy массив
                
                # Изменяем размер маски под размер оригинального изображения
                mask_i_resized = cv2.resize(mask_i, (width, height), interpolation=cv2.INTER_LINEAR)
                
                # Накладываем маску на пустую маску (255 для белого)
                mask[mask_i_resized > 0] = 255

    return mask

In [17]:
results = infer_image("./cv_test_dataset/test_img/_08.jpg")
mask_image = create_mask("./cv_test_dataset/test_img/_08.jpg", results)

# Сохраняем маску в формате PNG
mask_output_path = './mask_image.png'  # Укажите путь для сохранения маски
cv2.imwrite(mask_output_path, mask_image)

WARNING ⚠️ 'source' is missing. Using 'source=/home/jovyan/.mlspace/envs/nmkarpov/lib/python3.10/site-packages/ultralytics/assets'.

image 1/2 /home/jovyan/.mlspace/envs/nmkarpov/lib/python3.10/site-packages/ultralytics/assets/bus.jpg: 640x480 (no detections), 66.9ms


[ WARN:0@702.858] global loadsave.cpp:241 findDecoder imread_('./cv_test_dataset/test_img/_08.jpg'): can't open/read file: check file path/integrity


image 2/2 /home/jovyan/.mlspace/envs/nmkarpov/lib/python3.10/site-packages/ultralytics/assets/zidane.jpg: 384x640 3 contaminateds, 70.6ms
Speed: 2.1ms preprocess, 68.7ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)


[ WARN:0@703.063] global loadsave.cpp:241 findDecoder imread_('./cv_test_dataset/test_img/_08.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [24]:
model.save('/home/jovyan/nazar/123/123/misis_chill/nornikel_dockerfile/baseline3n.pt')

In [31]:
model_n = YOLO('/home/jovyan/nazar/123/123/misis_chill/nornikel_dockerfile/baseline3n.pt')
model_s = YOLO('/home/jovyan/nazar/123/123/misis_chill/nornikel_dockerfile/baseline2.pt')
model_n.to('cpu')
model_s.to('cpu')

YOLO(
  (model): SegmentationModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (m): ModuleList(
          (0): Bottleneck(
            (cv1): Conv(
              (conv): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (act): SiLU(inplace=True)
            )
            (cv2): Conv(
              (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (act): SiLU(inplace=True

In [ ]:
model_n.to('cpu')

In [38]:
model_s('/home/jovyan/nazar/123/123/misis_chill/train_dataset/baseline/datasets/train_data/images/train/1710275253_0.jpg')


image 1/1 /home/jovyan/nazar/123/123/misis_chill/train_dataset/baseline/datasets/train_data/images/train/1710275253_0.jpg: 384x640 2 contaminateds, 228.0ms
Speed: 2.4ms preprocess, 228.0ms inference, 3.9ms postprocess per image at shape (1, 3, 384, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: ultralytics.engine.results.Masks object
 names: {0: 'contaminated'}
 obb: None
 orig_img: array([[[ 21,  24,  32],
         [ 21,  24,  32],
         [ 22,  24,  32],
         ...,
         [ 94,  82,  64],
         [ 91,  79,  61],
         [ 97,  85,  67]],
 
        [[ 21,  24,  32],
         [ 21,  24,  32],
         [ 22,  24,  32],
         ...,
         [ 96,  84,  66],
         [ 92,  80,  62],
         [ 93,  81,  63]],
 
        [[ 22,  24,  32],
         [ 22,  24,  32],
         [ 22,  24,  32],
         ...,
         [ 97,  85,  67],
         [ 94,  82,  64],
         [ 95,  83,  65]],
 
        ...,
 
        [[140, 123, 102],
         [139, 123, 100],
         [133, 116,  95],
         ...,
         [ 67,  65,  47],
         [ 67,  64,  49],
         [ 67,  65,  47]],
 
        [[137, 122, 103],
         [136, 122, 100],
         [131, 11

In [39]:
model_n('/home/jovyan/nazar/123/123/misis_chill/train_dataset/baseline/datasets/train_data/images/train/1710275253_0.jpg')


image 1/1 /home/jovyan/nazar/123/123/misis_chill/train_dataset/baseline/datasets/train_data/images/train/1710275253_0.jpg: 384x640 1 contaminated, 83.3ms
Speed: 2.3ms preprocess, 83.3ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: ultralytics.engine.results.Masks object
 names: {0: 'contaminated'}
 obb: None
 orig_img: array([[[ 21,  24,  32],
         [ 21,  24,  32],
         [ 22,  24,  32],
         ...,
         [ 94,  82,  64],
         [ 91,  79,  61],
         [ 97,  85,  67]],
 
        [[ 21,  24,  32],
         [ 21,  24,  32],
         [ 22,  24,  32],
         ...,
         [ 96,  84,  66],
         [ 92,  80,  62],
         [ 93,  81,  63]],
 
        [[ 22,  24,  32],
         [ 22,  24,  32],
         [ 22,  24,  32],
         ...,
         [ 97,  85,  67],
         [ 94,  82,  64],
         [ 95,  83,  65]],
 
        ...,
 
        [[140, 123, 102],
         [139, 123, 100],
         [133, 116,  95],
         ...,
         [ 67,  65,  47],
         [ 67,  64,  49],
         [ 67,  65,  47]],
 
        [[137, 122, 103],
         [136, 122, 100],
         [131, 11